#### Notebook Summary as given by ChatGPT

- Presents a Number Type Theory that classifies all nonnegative integers by structural properties of their factorizations.
- Defines a total factorization function for n ≥ 0 and its inverse (factor_product), with tests illustrating the Fundamental Theorem of Arithmetic.
- Builds a hierarchy of predicates forming the Integer Type Tree (ITT) and implements a classifier that maps each n to one of 12 disjoint OEIS leaf sequences.
- Generates core and derived sequences, prints samples, and provides frequency counters and simple grid visualizations.
- Includes an OEIS search helper for sequence identification and an appendix with a LaTeX/TikZ diagram of the ITT for reference.

<h3 style="color:#CD5C5C;background:white; line-height: 150%;border-top: thick solid #CD5C5C; float: left; width: 100%; margin-top: 1em;">
Peter Luschny - October 2025

##Avant-propos

When I'm asked for a quick, easy introduction to combinatorics, I often refer to Gian-Carlo Rota's 'Twelvefold Way' (popularized by Richard Stanley's *Enumerative Combinatorics*). The Twelvefold Way is a systematic classification of 12 related combinatorial problems, organized in a table with 12 entries based on restrictions on the number and the color of balls in urns.

However, when I am faced with a similar question targeting elementary number theory, I can't point to anything comparable. 
Writing something similar to the TW was one of my motivations for this notebook. It was obvious to choose the Fundamental Theorem of Arithmetic as the starting point. Here we introduce 11 predicates that describe natural numbers solely in terms of their prime factorization.

To achieve a systematic representation, we will ensure that these predicates are mutually orthogonal. Thereby, every natural number is an element of precisely one of the 12 classes, and the union of these classes will encompass all natural numbers. This representation can then be clearly displayed as a binary tree. Interestingly, of the 23 nodes of this tree, 10 were added to OEIS only in the last 10 years.

In this notebook, we introduce the concept of factorization for all integers $ n \geq 0 $ and will adopt a fundamentally different primary classification than the one commonly used in the OEIS.

<h1 style="color:#CD5C5C;background:white; line-height: 150%;border-top: thick solid #CD5C5C; float: left; width: 100%; margin-top: 1em;">
Number Type Theory
</h1>

#### A taxonomy of integers from structural properties derived from the prime factorization.

## Factorization

Prime factorization is only defined for integers $ n \geq 2 $. 

- 1 has no prime factorization because 1 is not divisible by any prime.
- 0 has no prime factorization because 0 is divisible by every prime, so the factorization is not uniquely defined.

Informally, they describe the case "divisible by none" (for 1) and the case "divisible by all" (for 0).
We find it convenient to use the phrase **"n has no prime factors"** to describe both cases, although the phrases above are more precise.
In our setup, a factorization is always a **finite and nonempty list**, the exact form of which we will describe below.

However, we do not want to treat the two special cases, 0 and 1 separately in all our definitions and theorems, or to exclude them altogether. In fact, it is simple to include them by defining their factorizations as a list of tuples, like in the general case, call this the **factorization of n** for all integers $n \geq 0$, and reserve the term **prime factorization** if we implicitly assume $n \geq 2$.

### Factorization Definition

In this notebook, $\mathbb{N}$ denotes the nonnegative numbers, and we call this set the *natural numbers*. 
We will consider a function with signature (in Python notation)

* f(n: int) -> list[tuple[int, int]]
* Input: an integer $n \in \mathbb{N}$.
* Output: a finite list of pairs of natural numbers.

So if $f(n)$ is the output, then
$$
f(n) = \big[ (a_1, b_1), (a_2, b_2), \dots, (a_k, b_k) \big],
$$
for some $k \ge 0$, with each $(a_i, b_i) \in \mathbb{N} \times \mathbb{N}$.

Since $k$ depends on $n$, the codomain is the **set of all finite sequences** over $\mathbb{N} \times \mathbb{N}$:
$$
(\mathbb{N} \times \mathbb{N})^* := \bigcup_{k \in \mathbb{N}} (\mathbb{N} \times \mathbb{N})^k.
$$
The notation $X^*$ (Kleene star) denotes the set of all finite sequences over $X$. Thus we have formally:
$$
f : \mathbb{N} \to (\mathbb{N} \times \mathbb{N})^*
$$
$$
f(n) \in (\mathbb{N} \times \mathbb{N})^* \quad\text{for all } n \in \mathbb{N}.
$$
This can be formulated equivalently: "$f$ is a function that assigns to each natural number $n$ a finite sequence of pairs of natural numbers." 
Such a function is the function **factorization**.


In [1]:
def factorization(n: int) -> list[tuple[int, int]]:
    """
    Return list of (p, e) for n >= 0 in increasing p.
    """
    # Special case: 0 and 1 are represented as [(n, 0)]
    if n < 2: return [(n, 0)]

    x = n          # Working copy of n to factor
    fact = []      # List to store (prime, exponent) pairs
    d = 2          # Start with smallest prime candidate

    # Trial division up to sqrt(x)
    while d * d <= x:
        if x % d == 0:  # d is a factor
            e = 0       # Count exponent of this prime factor
            while x % d == 0:  # Extract all powers of d
                x //= d
                e += 1
            fact.append((d, e))  # Store (prime, exponent)
        # Increment: first 2 to 3, then by 2s (skip even numbers)
        d += 1 if d == 2 else 2
    
    # If x > 1 after trial division, it's a remaining prime factor
    if x > 1:
        fact.append((x, 1))

    return fact

Note that 'factorization' always returns a nonempty list of pairs of natural numbers. 
Thus if 
<pre>
* factorization(n) = [(f1, e1), (f2, e2), ..., (fk, ek)], 
</pre>

then $k \geq 1$ for all $n \geq 0$. Accordingly, the two **projections**, 

<pre>
* factors(n)   := [f for (f, _) in factorization(n)] and 
* exponents(n) := [e for (_, e) in factorization(n)].
</pre>

Both are nonempty lists. No conventions are needed regarding the product, sum, or other operations in the case of an empty set to define the factorization of all natural numbers. The representations of 0 and 1 preserve the uniqueness.

In [2]:
for n in range(11): print(n, '→', factorization(n))

0 → [(0, 0)]
1 → [(1, 0)]
2 → [(2, 1)]
3 → [(3, 1)]
4 → [(2, 2)]
5 → [(5, 1)]
6 → [(2, 1), (3, 1)]
7 → [(7, 1)]
8 → [(2, 3)]
9 → [(3, 2)]
10 → [(2, 1), (5, 1)]


At this point, we need to take a closer look at the cases n = 0 and n = 1. In connection with the multiple applications of multiplication, the natural numbers fall into two classes: the numbers in one class are immune to any change, in contrast the others change with each step. For example, 
<pre>
* [0^n | n >= 1] = 0, 0, 0,  0,  0,  0,   0, ...
* [1^n | n >= 1] = 1, 1, 1,  1,  1,  1,   1, ...
* [2^n | n >= 1] = 2, 4, 8, 16, 32, 64, 128, ....
</pre>
Because of this remarkable property of 0 and 1, these numbers are called the *idempotent numbers*.
The exponent of these numbers (and only of these numbers) in the factorization is 0; all other factors have an exponent that is at least 1.

### Inverse of Factorization

Since each number has a unique factorization (which of course needs to be proven), we can consider its inverse function.

In [3]:
from math import prod

def factor_product(factors: list[tuple[int, int]]) -> int:
    """ Returns the product of the factors. Uses the
    modified power function: f^m if m >= 2, otherwise f."""
    return prod(f**m if m > 1 else f for (f, m) in factors)

The factor product is based on the *multiplicity function* 
$$ \mathbb{N} \times \mathbb{N} \rightarrow \mathbb{N}, \ (f, m) \rightarrow f^m \ \text{ if } m \ge 2, \text{otherwise } f. $$
The multiplicity function treats idempotent numbers separately, assigning them to themselves as their own powers. This function belongs to the family of the power function and is context-sensitive. One should not assume that this definition can be applied automatically to another subject area; for example, one usually chooses a different variant in combinatorics. 

The function *factor_product* reconstructs the integer from its factorization. It is the **inverse** of the factorization function, meaning that **factor_product(factorization(n)) = n** for all $n \geq 0$. In mathematics, this identity is called the **Fundamental Theorem of Arithmetic**. What makes this theorem fundamental are two facts:
- One fact is that for $n \geq 2$, the first component of every tuple in every list is a prime, and eventually all primes will appear this way. Note that we have not presupposed the notion of a 'prime' in the definition of the factorization or its inverse, so this is a non-trivial fact. 

We can use this fact to *define* the notion of a prime: An integer $p$ is prime if and only if its factorization is [(p, 1)]. We adopt this as our definition of a prime number.
- Another fact, and this is where the depth of the theorem resides, is that the prime factorization is **unique**. This means that no other list of tuples with primes in the first component that is ordered in lexicographical order will reconstruct the original integer.
 
The representation given here is valid for all $ n \geq 0 $ and smoothly integrates the cases $ n = 0 $ and $ n = 1 $. We do *not* use the convention for products over empty sets; instead, we rely on explicit definitions.

In [4]:
# When executing this test function you will only see some output if the test fails.

def FTA_test(lng: int) -> None:
    """Test the FTA (Fundamental Theorem of Arithmetic)"""
    for n in range(lng):
        f = factorization(n)
        p = factor_product(f)
        # Raise an assertion error if not equal
        assert n == p, "n={} f={} p={}".format(n, f, p)

FTA_test(100)

Some CAS systems completely exclude the case 0 and generate an error halt; others return an empty list for the cases $ n = 0 $ and $ n = 1 $, so these cases cannot be distinguished by their return value and must be handled explicitly.

### Predicates for Integer Types

We can classify integers according to structural properties derived from their  factorization. 
We have already seen two examples: **idempotent numbers** are numbers whose factorization is [(n, 0)];
**prime numbers** are numbers whose factorization is [(n, 1)]. We can define many more integer types in this way. For example, we define the type of **square numbers** as numbers whose prime factorization has only even exponents.

In principle, properties of integers should be defined directly by the properties of their factorization. We have a foundation (FTA), so let's build on it. 

For example, what is a composite number?
The popular website MathWorld says: "A composite number n is a positive integer n>1 which is not prime (i.e., which has factors other than 1 and itself)."
This is cumbersome and indirect. First, the concept of 'prime' is introduced, then it is negated, then 1 is excluded, and the case of 0 is not considered from the outset. This does not reflect conceptual thinking. A simple way to define a composite number is: A number is called **composite** if the sum of the exponents 
in its factorization is greater than or equal to 2. This covers everything, including the case of 0, and it satisfies our intuition: you need at least two things to make a composite number.

In the following, we have selected 11 predicates in this spirit based solely on factorization and will place them in a systematic context. Each predicate divides the set of integers into two disjoint subsets depending on whether the predicate applies to the terms or not. 

<center>

|split | predicate| split| predicate|
|--|--|--|--|
|$ A \rightarrow \, B $ | $ max\ exps = 0 $     | $ A \rightarrow \, C $ | $ min\ exps \ge 1 $ |
|$ C \rightarrow \, D $ | $ primes\ count = 1 $ | $ C \rightarrow \, K $ | $ primes\ count \ge 2 $ |
|$ D \rightarrow \, E $ | $ max\ exps = 1 $     | $ D \rightarrow \, H $ | $ max\ exps \ge 2 $ |
|$ K \rightarrow \, L $ | $ all\ exps\ equal $  | $ K \rightarrow \, S $ | $ not\ all\ exps\ equal $ |
|$ E \rightarrow \, F $ | $ n \equiv 1\ mod\ 4 $ | $ E \rightarrow \, G $ | $ n \equiv 2,3\ mod\ 4 $ |
|$ H \rightarrow \, I $ | $ max\ exps = 2 $     | $ H \rightarrow \, J $ | $ max\ exps > 2 $ |
|$ L \rightarrow \, M $ | $ max\ exps = 1 $     | $ L \rightarrow \, P $ | $ max\ exps \ge 2 $ |
|$ S \rightarrow \, X $ | $ gcd\ exps \ge 2 $   | $ S \rightarrow \, T $ | $ gcd\ exps = 1 $ |
|$ M \rightarrow \, N $ | $ primes\ count = 2 $ | $ M \rightarrow \, O $ | $ primes\ count \ge 3 $ |
|$ P \rightarrow \, Q $ | $ max\ exps = 2 $     | $ P \rightarrow \, R $ | $ max\ exps > 2 $ |
|$ T \rightarrow \, U $ | $ min\ exps > 1 $     | $ T \rightarrow \, W $ | $ min\ exps = 1 $ |

</center>

By hierarchically combining these predicates, we obtain a tree-based classification of integers, the **Integer Type Tree (ITT)**. The inner nodes represent the splitting of the sets, while *the 12 leaves form a complete disjoint decomposition of the root* set, which is the set of the natural numbers including 0.

### An Integer Type Tree 
<center>
<img src="ITTree.png" width="720" height="500">
</center>

### The basic trisection

A typology such as the one above is, of course, not unique.
In the OEIS, the basic classification is usually described as follows.

The positive numbers 1, 2, ... are divided into three sets: 

- the unit {1}, 
- the primes, and 
- numbers with at least 2 factors > 1;
 
i.e. A000027 = {1} U A000040  U A002808.

In contrast, the top of our tree says:

The natural numbers 0, 1, 2, ... are divided into three sets: 
- the idempotent numbers {0, 1}, 
- the prime powers, and 
- numbers with at least 2 distinct prime factors; 

i.e. A001477 = A387487 U A246655 U A024619.

This alternative view emphasizes the special roles of 0 and 1 while highlighting their commonality, rather than their differences. While the third requirement in conventional classification demands the existence of at least two factors > 1, we require at least two distinct prime factors.

### Illustrating the integration of the zero

An integer n is 'squarefree' if n is not divisible by a square greater than 1. Thus, 0 is not squarefree because 0 is divisible by 4, and 1 is squarefree, by special arrangement. Obviously, the definition of a squarefree number is arbitrary and convoluted.

MathWorld mentions "technical reasons" why Mathematica considers 1 to be squarefree, being a 'convention', and makes fun of it: "The number 1 therefore has the somewhat curious distinction of being simultaneously a perfect square and squarefree."

This is all confusing moonshine and contradicts mathematicians' ideal of building a consistent and beautiful theory. This is all the more regrettable at a time when the formalization of mathematical statements is already taught in introductory university courses (Kontorovich/Tao) and the Lean-based mathlib has become the standard reference for young mathematicians.

To illustrate how the integration of 0 works in our framework, we consider the definition of the notion 'kernel of a number' and the predicate 'is flat'. The 'kernel' of a number is the product of its distinct factors (multiplicities are ignored). If the kernel of n coincides with n, then we call n 'flat'. I.e., the flat numbers are the fixed points of the 'kernel' function. This is the simple formal statement, free of any quirks, and valid for all $n \geq 0$: 

In [ ]:
from math import prod

# The kernel for n >= 0
def kernel(n: int) -> int:
    return prod(f for (f, _) in factorization(n))

def is_flat(n: int) -> bool: 
    return n == kernel(n)

The terms *is_flat* and *is_squarefree* are the same except for 0: *is_squarefree* excludes 0, while *is_flat* includes it. But that is not our view! 
For us, the 'real' squarefree numbers are A144338, that is, the 'nontrivial products of distinct primes'. And *is_flat* adds the idempotent numbers A387487 to these, because these are incapable of being raised to any real power, they are flat by nature.

In any case, we can offer 'n is flat' as a complete replacement for 'n is squarefree', provided $n \geq 2$. And since we only develop the right child of the root in our tree, we can do so safely in the following. Parenthetically, one could also use the word 'scenic' for 'nonflat'.

The definition of 'kernel' and 'is_flat' concerns the *first projection* of the factorization of n, binding them together by *multiplication*.
The functions 'exp_sum' and 'is_prime' are the counterparts using the *second projection* of the factorization of n, binding them together by *summation*.

In [6]:
# The sum of multiplicities, for n >= 0.
def exp_sum(n: int) -> int:
    return sum(e for (_, e) in factorization(n))

def is_prime(n: int) -> bool: 
    return 1 == exp_sum(n)

These two examples demonstrate how simply and consistently number-theoretic concepts can be defined for all $n \geq 0$. The fact that they are often defined only for $n \geq 1$ is sometimes defended by the remark that arithmetic functions, by definition, are only defined for $n \geq 1$. But this quickly becomes a circular justification.

As a supplement, we add the definition of the *Möbius function* μ(n) for $n \geq 0$.
For $n \in \{0, 1\}$ the definition below gives μ(0) = 1, so the term 0 will be prepended to A030229.

In [7]:
def moebius(n: int) -> int:
    if is_flat(n): return 1 if exp_sum(n) % 2 == 0 else -1
    return 0

print([moebius(n) for n in range(24)])
print([n for n in range(61) if moebius(n) == 1],  "A030229")
print([n for n in range(61) if moebius(n) == -1], "A030059") 
print([n for n in range(51) if moebius(n) == 0],  "A013929")
print([n for n in range(29) if moebius(n) <= 0],  "A390278") 
print([n for n in range(29) if moebius(n) >= 0],  "A174891")

[1, 1, -1, -1, 0, -1, 1, -1, 0, 0, 1, -1, 0, -1, 1, 1, 0, -1, 0, -1, 0, 1, 1, -1]
[0, 1, 6, 10, 14, 15, 21, 22, 26, 33, 34, 35, 38, 39, 46, 51, 55, 57, 58] A030229
[2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 30, 31, 37, 41, 42, 43, 47, 53, 59] A030059
[4, 8, 9, 12, 16, 18, 20, 24, 25, 27, 28, 32, 36, 40, 44, 45, 48, 49, 50] A013929
[2, 3, 4, 5, 7, 8, 9, 11, 12, 13, 16, 17, 18, 19, 20, 23, 24, 25, 27, 28] A390278
[0, 1, 4, 6, 8, 9, 10, 12, 14, 15, 16, 18, 20, 21, 22, 24, 25, 26, 27, 28] A174891


Developing an alternative integer typology based on Moebius's trisection appears to be an appealing project. 


### Two dictionaries to describe the A-numbers and the nodes

Although we have fully described our program in the predicate table and the tree diagram above, we still need to convert it into a computer-readable representation. In addition to the names (which are the A-numbers of the OEIS and serve as the keys in the dictionary), we add a brief description of each sequence.

In [8]:
# A table with A-numbers and short descriptions
# Mapping of A-numbers to (parent A-number, description, node_label, predicate)
# Descriptions are short phrases, not complete sentences.

SEQUENCES: dict[str, tuple[str, str, str, str]] = { 
    "A001477": ("ROOT",    "Natural Numbers (nonnegative integers)", 'A', "n >= 0"),
    "A387487": ("A001477", "{0, 1}, idempotent numbers, no prime factor", 'B', "max exps = 0"),
    "A020725": ("A001477", "Has at least one prime factor, integers >= 2", 'C', "min exps >= 1"),
    "A246655": ("A020725", "All prime powers (p^k, k >= 1)", 'D', "# distinct primes = 1"),
    "A024619": ("A020725", "At least 2 distinct prime factors", 'K', "# distinct primes >= 2"),
    "A000040": ("A246655", "Primes, one factor with multiplicity = 1", 'E', "max exps = 1"),
    "A246547": ("A246655", "Proper prime powers (p^k, k >= 2)", 'H', "max exps >= 2"),
    "A182853": ("A024619", "Distinct factors > 1, all multiplicities = 1", 'L', "all exps = 1"),
    "A059404": ("A024619", "Different multiplicities", 'S', "exps not all equal"),
    "A120944": ("A182853", "Composite flat numbers", 'M', "flat composites"),
    "A303606": ("A182853", "Equi-powered composites, all multiplicities >= 2", 'P', "min exps >= 2"),
    "A177492": ("A303606", "Products of squares of 2 or more distinct primes", 'Q', "all exps = 2"),
    "A388304": ("A303606", "Prime factors > 1, all exps are equal and > 2", 'R', "all exps > 2 and equal"),
    "A303946": ("A059404", "Neither flat nor perfect powers", 'T', "gcd exps = 1"),
    "A389864": ("A059404", "Perfect powers with mixed multiplicity", 'X', "gcd exps >= 2"),
    "A002144": ("A000040", "Primes = 1 (mod 4) (Pythagorean primes)", 'F', "n ≡ 1 (mod 4)"),
    "A045326": ("A000040", "Primes = 3 (mod 4), or 2", 'G', "n ≡ 2,3 (mod 4)"),
    "A001248": ("A246547", "Squares of primes", 'I', "max exps = 2"),
    "A246549": ("A246547", "Higher prime powers (cubes, fourth powers, ...)", 'J', "max exps >= 3"),
    "A006881": ("A120944", "Product of two distinct primes", 'N', "# distinct primes = 2"),
    "A350352": ("A120944", "Products of >= 3 distinct primes", 'O', "# distinct primes >= 3"),
    "A052486": ("A303946", "Powerful but not perfect (Achilles numbers)", 'U', "min exps > 1"),
    "A332785": ("A303946", "Mixed prime powers: some multpl >= 2, some = 1", 'W', "min exps = 1"),
    "A144338": ("-",       "Products of distinct primes", '-', "-"),
    "A072777": ("-",       "Powers of products of distinct primes", '-', "-"),
    "A328956": ("-",       "sigma = omega * bigomega", '-', "-"),
}

# Helper functions to access the dictionary
def describe(anum: str) -> str:
    """Return short description for A-number or empty string if unknown."""
    val = SEQUENCES.get(anum)
    if val is not None:
        return val[1]  # description is the second element
    return "? not known"

def get_parent(anum: str) -> str:
    """Return parent A-number for given A-number."""
    val = SEQUENCES.get(anum)
    if val is not None:
        return val[0]  # parent is the first element
    return "? not known"

def get_node_label(anum: str) -> str:
    """Return node label for given A-number."""
    val = SEQUENCES.get(anum)
    if val is not None:
        return val[2]  # node label is the third element
    return "?"

def get_predicate(anum: str) -> str:
    """Return predicate string for given A-number (fourth tuple element)."""
    val = SEQUENCES.get(anum)
    if val is not None:
        return val[3]
    return "? not known"

In [9]:
# Node label to predicate mapping
NODE_PREDICATES: dict[str, str] = {
    'A': "n >= 0",
    'B': "max exps = 0",
    'C': "min exps >= 1",
    'D': "# distinct primes = 1",
    'E': "max exps = 1",
    'F': "n ≡ 1 (mod 4)",
    'G': "n ≡ 2,3 (mod 4)",
    'H': "max exps >= 2",
    'I': "max exps = 2",
    'J': "max exps >= 3",
    'K': "# distinct primes >= 2",
    'L': "all exps = 1",
    'M': "flat composites",
    'N': "# distinct primes = 2",
    'O': "# distinct primes >= 3",
    'P': "min exps >= 2",
    'Q': "all exps = 2",
    'R': "all exps > 2 and equal",
    'S': "exps not all equal",
    'T': "gcd exps = 1",
    'U': "min exps > 1",
    'W': "min exps = 1",
    'X': "gcd exps >= 2",
}

# Optional helper
def get_predicate_by_label(label: str) -> str:
    return NODE_PREDICATES.get(label, "? not known")

## The Classifier

The following function is the main function and could also be called the tree builder for the tree shown above. It assigns each integer $n \geq 0$ to one of 12 mutually disjoint classes whose union is $\mathbb{N}$. Apart from the functions 'factorization' and 'gcd' it does not use any external functions.

In [10]:
from math import gcd
from functools import reduce

def classifier(n: int) -> str:
    """
    Return the A-number of the sequence of which the given
    nonnegative integer n is a term (leaf in the tree).
    """

    # idempotents, n has no prime factors
    if n in (0, 1):
        return "A387487"  # idempotents 0 and 1

    # now prime factorization applies, i.e. n >= 2
    factors = factorization(n)  # list of (prime, exponent)

    primes_count = len(factors)
    exps = [e for (_, e) in factors]
    max_e = max(exps)
    min_e = min(exps)

    # n has at least one prime factor
    if primes_count == 1:
        if max_e == 1:
            return "A002144" if n % 4 == 1 else "A045326"
        else: # prime powers
            return "A001248" if max_e == 2 else "A246549"

    all_exps_equal = (len(set(exps)) == 1)
    # composite, multi-prime
    if all_exps_equal:
        if max_e == 1: # flat composites
            return "A006881" if primes_count == 2 else "A350352"
        else: # all exponents equal and ≥ 2
            return "A177492" if max_e == 2 else "A388304"

    # varied exponents
    g = reduce(gcd, exps)
    if g >= 2:
        return "A389864"
    else:
        return "A052486" if min_e > 1 else "A332785"

    # return "This can never happen!"  # sanity check

In [11]:
# Demonstration of the classifier

def classifier_demo(lng): 
    for n in range(lng):
        anum = classifier(n)
        node = get_node_label(anum)
        desc = describe(anum)
        print(f"{n:2d} → {anum} [{node}]  {desc}")

classifier_demo(33)

 0 → A387487 [B]  {0, 1}, idempotent numbers, no prime factor
 1 → A387487 [B]  {0, 1}, idempotent numbers, no prime factor
 2 → A045326 [G]  Primes = 3 (mod 4), or 2
 3 → A045326 [G]  Primes = 3 (mod 4), or 2
 4 → A001248 [I]  Squares of primes
 5 → A002144 [F]  Primes = 1 (mod 4) (Pythagorean primes)
 6 → A006881 [N]  Product of two distinct primes
 7 → A045326 [G]  Primes = 3 (mod 4), or 2
 8 → A246549 [J]  Higher prime powers (cubes, fourth powers, ...)
 9 → A001248 [I]  Squares of primes
10 → A006881 [N]  Product of two distinct primes
11 → A045326 [G]  Primes = 3 (mod 4), or 2
12 → A332785 [W]  Mixed prime powers: some multpl >= 2, some = 1
13 → A002144 [F]  Primes = 1 (mod 4) (Pythagorean primes)
14 → A006881 [N]  Product of two distinct primes
15 → A006881 [N]  Product of two distinct primes
16 → A246549 [J]  Higher prime powers (cubes, fourth powers, ...)
17 → A002144 [F]  Primes = 1 (mod 4) (Pythagorean primes)
18 → A332785 [W]  Mixed prime powers: some multpl >= 2, some = 1


This was essentially what I wanted to write. Below there is a lot of technical stuff, but mainly for verification. So you may stop reading here.

The classifier function in Lean 4 is shown in the cell below, which I put in 'raw' mode so that it doesn't interfere with Python execution. Accordingly, it is also not displayed on GitHub.

### The classification certificate: the classifier_path function

ChatGPT wrote the functions in the cell below based on the tree description and the predicates given above. It shows the decision path through the tree for each number n. You can choose to display the path in four different styles: 'letters', 'mini', 'labels', and 'verbose'. There is no new mathematical content in this function compared to what was said above. But it not only helps to visualize the classification. In fact it is a full **classification certificate** for each integer.

In [12]:
PATH_LABELS_OF_LEAF = {
    'A001477': ['A'],
    'A387487': ['A', 'B'],
    'A002144': ['A', 'C', 'D', 'E', 'F'],
    'A045326': ['A', 'C', 'D', 'E', 'G'],
    'A001248': ['A', 'C', 'D', 'H', 'I'],
    'A246549': ['A', 'C', 'D', 'H', 'J'],
    'A006881': ['A', 'C', 'K', 'L', 'M', 'N'],
    'A350352': ['A', 'C', 'K', 'L', 'M', 'O'],
    'A177492': ['A', 'C', 'K', 'L', 'P', 'Q'],
    'A388304': ['A', 'C', 'K', 'L', 'P', 'R'],
    'A052486': ['A', 'C', 'K', 'S', 'T', 'U'],
    'A332785': ['A', 'C', 'K', 'S', 'T', 'W'],
    'A389864': ['A', 'C', 'K', 'S', 'X']
 }

In [13]:
# Path tracing through the Integer Type Tree
# style: 'letters' | 'mini' | 'verbose' | 'labels'
# - letters: A → C → D → E → F
# - mini:    A [Axxxxx] → C [Ayyyyy] → ... (OEIS A-numbers per node, no final leaf A-number)
# - verbose: First line prints "n → <leaf A-number>", followed by one line per node as "    Label — Description" (no arrows)
# - labels:  A → C → D → E → F (no A-number suffix)

# Build node-label → A-number mapping from SEQUENCES
NODE_TO_ANUM: dict[str, str] = {}
for anum, (parent, desc, label, predicate) in SEQUENCES.items():
    if label != '-' and label not in NODE_TO_ANUM:
        NODE_TO_ANUM[label] = anum

# 'mini' tags now show OEIS A-numbers for each node label
MINI_TAGS: dict[str, str] = {label: anum for label, anum in NODE_TO_ANUM.items()}

# Standard indent for verbose output: exactly 4 spaces
INDENT = "    "


def _format_step(label: str, style: str) -> str:
    if style in ('letters', 'labels'):
        return label
    anum = NODE_TO_ANUM.get(label)
    desc = describe(anum) if anum else ''
    if style == 'mini':
        tag = MINI_TAGS.get(label) or (desc.split(',')[0] if desc else label)
        return f"{label} [{tag}]"
    if style == 'verbose':
        # Caller handles indentation and joining for verbose style
        return f"{label} — {desc}" if desc else label
    return label


def classifier_path(
    n: int,
    style: str = 'letters',
    include_anum: bool = True,
    show: bool = False,
    header: bool = True,
    verbose_use_predicate: bool = False,
) -> str | None:
    """
    Unified path renderer/printer.

    Parameters
    - n: integer to classify
    - style: 'letters' | 'mini' | 'verbose' | 'labels'
    - include_anum: when True, append leaf (A-number) suffix on single-line styles
    - show: when True, print to stdout; otherwise return string
    - header: for verbose style, include the header line "n → <leaf A-number>"
    - verbose_use_predicate: when True, verbose shows get_predicate(anum) instead of describe(anum)

    Behavior by style
    - labels: bare labels; no leaf suffix
    - mini: labels with [A-numbers] per node; no leaf suffix
    - letters: bare labels with optional leaf suffix when include_anum=True
    - verbose: if show=True prints header (if header=True) then INDENT + "Label — Text" per line;
               when show=False returns the same multi-line text as a string
               where Text = describe(anum) unless verbose_use_predicate=True

    Implementation notes
    - Path labels are sourced from PATH_LABELS_OF_LEAF[classifier(n)]
    """
    # 'labels' and 'mini' styles suppress the final leaf A-number suffix
    if style in ('labels', 'mini'):
        include_anum = False

    # Determine leaf A-number via classifier and fetch the path labels
    leaf = classifier(n)
    labels = PATH_LABELS_OF_LEAF.get(leaf, [])

    if style == 'verbose':
        lines: list[str] = []
        for label in labels:
            anum = NODE_TO_ANUM.get(label) 
            text = get_predicate(anum) if verbose_use_predicate else (describe(anum) if anum else '')
            lines.append(f"{INDENT}{label} — {text}" if text else f"{INDENT}{label}")
        body = "\n".join(lines)
        if show:
            if header:
                print(f"{n} → {leaf}")
            print(body)
            return None
        # return string
        return (f"{n} → {leaf}\n" + body) if header else body

    # Single-line styles
    formatted = [ _format_step(lbl, style) for lbl in labels ]
    line = " → ".join(formatted)
    if include_anum:
        line = f"{line} ({leaf})"

    if show:
        print(f"{n} → {line}")
        return None
    return line

In [14]:
# Demo for various styles of classifier_path
for n in [0, 1, 2, 4, 5, 6, 8, 12, 30, 36, 72, 144, 216]:
    classifier_path(n, style='labels', show=True)
    classifier_path(n, style='mini', show=True)
    classifier_path(n, style='verbose', show=True)
    print()

0 → A → B
0 → A [A001477] → B [A387487]
0 → A387487
    A — Natural Numbers (nonnegative integers)
    B — {0, 1}, idempotent numbers, no prime factor

1 → A → B
1 → A [A001477] → B [A387487]
1 → A387487
    A — Natural Numbers (nonnegative integers)
    B — {0, 1}, idempotent numbers, no prime factor

2 → A → C → D → E → G
2 → A [A001477] → C [A020725] → D [A246655] → E [A000040] → G [A045326]
2 → A045326
    A — Natural Numbers (nonnegative integers)
    C — Has at least one prime factor, integers >= 2
    D — All prime powers (p^k, k >= 1)
    E — Primes, one factor with multiplicity = 1
    G — Primes = 3 (mod 4), or 2

4 → A → C → D → H → I
4 → A [A001477] → C [A020725] → D [A246655] → H [A246547] → I [A001248]
4 → A001248
    A — Natural Numbers (nonnegative integers)
    C — Has at least one prime factor, integers >= 2
    D — All prime powers (p^k, k >= 1)
    H — Proper prime powers (p^k, k >= 2)
    I — Squares of primes

5 → A → C → D → E → F
5 → A [A001477] → C [A020725] → 

In [15]:
# ... or in expert mode showing the predicates instead of descriptions:

for n in [0, 1, 2, 4, 5, 6, 8, 12, 30, 36, 72, 144, 216]:
    classifier_path(n, style='verbose', verbose_use_predicate=True, show=True)
    print()

0 → A387487
    A — n >= 0
    B — max exps = 0

1 → A387487
    A — n >= 0
    B — max exps = 0

2 → A045326
    A — n >= 0
    C — min exps >= 1
    D — # distinct primes = 1
    E — max exps = 1
    G — n ≡ 2,3 (mod 4)

4 → A001248
    A — n >= 0
    C — min exps >= 1
    D — # distinct primes = 1
    H — max exps >= 2
    I — max exps = 2

5 → A002144
    A — n >= 0
    C — min exps >= 1
    D — # distinct primes = 1
    E — max exps = 1
    F — n ≡ 1 (mod 4)

6 → A006881
    A — n >= 0
    C — min exps >= 1
    K — # distinct primes >= 2
    L — all exps = 1
    M — flat composites
    N — # distinct primes = 2

8 → A246549
    A — n >= 0
    C — min exps >= 1
    D — # distinct primes = 1
    H — max exps >= 2
    J — max exps >= 3

12 → A332785
    A — n >= 0
    C — min exps >= 1
    K — # distinct primes >= 2
    S — exps not all equal
    T — gcd exps = 1
    W — min exps = 1

30 → A350352
    A — n >= 0
    C — min exps >= 1
    K — # distinct primes >= 2
    L — all exps = 

## Generating sequences

In [16]:
# Generate all tree leaves quickly in one pass

Anames = ['A387487', 'A002144', 'A045326', 'A001248', 'A246549', 'A006881', 
          'A350352', 'A389864', 'A332785', 'A052486', 'A388304', 'A177492']
Aseq = {aname: [] for aname in Anames}

def show_classification(search_length: int, display_length: int) -> None:
    for n in range(search_length):
        anum = classifier(n)
        if anum in Aseq:
            Aseq[anum].append(n)
        else:
            print("UHUH...", n, anum)

    print("*** The leaves of the type tree are:\n")
    for anum in Anames:
        print(anum, [get_node_label(anum)], describe(anum), '\n', 
              Aseq[anum][:display_length])

show_classification(11000, 20)

*** The leaves of the type tree are:

A387487 ['B'] {0, 1}, idempotent numbers, no prime factor 
 [0, 1]
A002144 ['F'] Primes = 1 (mod 4) (Pythagorean primes) 
 [5, 13, 17, 29, 37, 41, 53, 61, 73, 89, 97, 101, 109, 113, 137, 149, 157, 173, 181, 193]
A045326 ['G'] Primes = 3 (mod 4), or 2 
 [2, 3, 7, 11, 19, 23, 31, 43, 47, 59, 67, 71, 79, 83, 103, 107, 127, 131, 139, 151]
A001248 ['I'] Squares of primes 
 [4, 9, 25, 49, 121, 169, 289, 361, 529, 841, 961, 1369, 1681, 1849, 2209, 2809, 3481, 3721, 4489, 5041]
A246549 ['J'] Higher prime powers (cubes, fourth powers, ...) 
 [8, 16, 27, 32, 64, 81, 125, 128, 243, 256, 343, 512, 625, 729, 1024, 1331, 2048, 2187, 2197, 2401]
A006881 ['N'] Product of two distinct primes 
 [6, 10, 14, 15, 21, 22, 26, 33, 34, 35, 38, 39, 46, 51, 55, 57, 58, 62, 65, 69]
A350352 ['O'] Products of >= 3 distinct primes 
 [30, 42, 66, 70, 78, 102, 105, 110, 114, 130, 138, 154, 165, 170, 174, 182, 186, 190, 195, 210]
A389864 ['X'] Perfect powers with mixed multiplicit

In [17]:
def internal_nodes(max: int) -> None:
    """Generate and print sequences up to max items each."""

    print("\n*** The internal nodes of the type tree are:\n")

    A000040 = sorted([*Aseq['A002144'], *Aseq['A045326']])
    print('A000040', ["E"], describe('A000040'), '\n', A000040[:max])

    A246547 = sorted([*Aseq['A001248'], *Aseq['A246549']])
    print('A246547', ["H"], describe('A246547'), '\n', A246547[:max])

    A120944 = sorted([*Aseq['A006881'], *Aseq['A350352']])
    print('A120944', ["M"], describe('A120944'), '\n', A120944[:max])

    A303946 = sorted([*Aseq['A052486'], *Aseq['A332785']])
    print('A303946', ["T"], describe('A303946'), '\n', A303946[:max])

    A303606 = sorted([*Aseq['A177492'], *Aseq['A388304']])
    print('A303606', ["P"], describe('A303606'), '\n', A303606[:max])

    A059404 = sorted([*A303946, *Aseq['A389864']])
    print('A059404', ["S"], describe('A059404'), '\n', A059404[:max])

    A182853 = sorted([*A120944, *A303606])
    print('A182853', ["L"], describe('A182853'), '\n', A182853[:max])

    A024619 = sorted([*A182853, *A059404])
    print('A024619', ["K"], describe('A024619'), '\n', A024619[:max])

    A246655 = sorted([*A000040, *A246547])
    print('A246655', ["D"], describe('A246655'), '\n', A246655[:max])

    A020725 = sorted([*A246655, *A024619])
    print('A020725', ["C"], describe('A020725'), '\n', A020725[:max])

    #A328956 = sorted([*Aseq['A006881'], *Aseq['A332785']])  # assert!
    #print('A328956', [" "], describe('A328956'), '\n', A328956[:max])
    #A072777 = sorted([*A246547, *A303606])
    #print('A072777', [" "], describe('A072777'), '\n', A072777[:max])

internal_nodes(15)


*** The internal nodes of the type tree are:

A000040 ['E'] Primes, one factor with multiplicity = 1 
 [2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47]
A246547 ['H'] Proper prime powers (p^k, k >= 2) 
 [4, 8, 9, 16, 25, 27, 32, 49, 64, 81, 121, 125, 128, 169, 243]
A120944 ['M'] Composite flat numbers 
 [6, 10, 14, 15, 21, 22, 26, 30, 33, 34, 35, 38, 39, 42, 46]
A303946 ['T'] Neither flat nor perfect powers 
 [12, 18, 20, 24, 28, 40, 44, 45, 48, 50, 52, 54, 56, 60, 63]
A303606 ['P'] Equi-powered composites, all multiplicities >= 2 
 [36, 100, 196, 216, 225, 441, 484, 676, 900, 1000, 1089, 1156, 1225, 1296, 1444]
A059404 ['S'] Different multiplicities 
 [12, 18, 20, 24, 28, 40, 44, 45, 48, 50, 52, 54, 56, 60, 63]
A182853 ['L'] Distinct factors > 1, all multiplicities = 1 
 [6, 10, 14, 15, 21, 22, 26, 30, 33, 34, 35, 36, 38, 39, 42]
A024619 ['K'] At least 2 distinct prime factors 
 [6, 10, 12, 14, 15, 18, 20, 21, 22, 24, 26, 28, 30, 33, 34]
A246655 ['D'] All prime powers (p^k, k 

In [18]:
def compute_sequence(Anum: str, lim: int) -> list[int]:
    n, s = 0, 0
    seq = []
    while n < 1000000000 and s < lim:
        if Anum == classifier(n):
            seq.append(n)
            s += 1
        n += 1
    # print(s, "terms found for", Anum, ", requested", lim, "reached:", s == lim)
    return seq

print(compute_sequence("A388304", 20))

[216, 1000, 1296, 2744, 3375, 7776, 9261, 10000, 10648, 17576, 27000, 35937, 38416, 39304, 42875, 46656, 50625, 54872, 59319, 74088]


### The distribution of integer types

In [19]:
def type_counter(lng: int) -> None:

    Anames = ['A332785', 'A350352', 'A006881', 'A045326', 'A002144', 'A052486',
              'A177492', 'A389864', 'A001248', 'A246549', 'A388304', 'A387487']
    Iseq = {aname: 0 for aname in Anames}

    for n in range(lng):
        anum = classifier(n)
        if anum in Iseq:
            Iseq[anum] += 1
        else:
            print("UHUH...", n, anum)

    print(f"\n*** Integers as distributed below {lng} among the disjoint core sequences:\n")
    for anum in Anames:
        print(f"{get_node_label(anum)}  {anum} {Iseq[anum]:6d} {100.0 * Iseq[anum] / lng:7.2f} %  {describe(anum)}")

# Run the counter
type_counter(100000)


*** Integers as distributed below 100000 among the disjoint core sequences:

W  A332785  38588   38.59 %  Mixed prime powers: some multpl >= 2, some = 1
O  A350352  27888   27.89 %  Products of >= 3 distinct primes
N  A006881  23313   23.31 %  Product of two distinct primes
G  A045326   4809    4.81 %  Primes = 3 (mod 4), or 2
F  A002144   4783    4.78 %  Primes = 1 (mod 4) (Pythagorean primes)
U  A052486    252    0.25 %  Powerful but not perfect (Achilles numbers)
Q  A177492    127    0.13 %  Products of squares of 2 or more distinct primes
X  A389864    109    0.11 %  Perfect powers with mixed multiplicity
I  A001248     65    0.07 %  Squares of primes
J  A246549     43    0.04 %  Higher prime powers (cubes, fourth powers, ...)
R  A388304     21    0.02 %  Prime factors > 1, all exps are equal and > 2
B  A387487      2    0.00 %  {0, 1}, idempotent numbers, no prime factor


*Amiram Eldar* mentions that asymptotically, only A332785 and A350352 have a positive density ($1-1/\zeta(2)$ and $1/\zeta(2)$, respectively), and all the rest have density 0.

Although, for example, the product of two distinct prime numbers below 100000 makes up almost 1/4 of all integers, their asymptotic density is 0. To see this consider
the set of integers that are the product of two **distinct** primes:
$$ A = \{pq : p<q\text{ primes}\}.$$
By Landau’s theorem we know that
$$A(x) \sim \frac{x\log\log x}{\log x}.$$
So
$$\frac{A(x)}{x} \sim \frac{\log\log x}{\log x}.$$
Since
$$\frac{\log\log x}{\log x} \longrightarrow 0,$$
that means: The asymptotic density of products of two distinct primes is 0.

In [20]:
# Visualize the different integer classes in a square grid.

def classifier_square(len: int):
    for n in range(len):
        s = []
        for k in range(len):
            s.append(get_node_label(classifier(n*len + k))) 
        print(s)

# Show an n X n square with the labels of the tree nodes.
classifier_square(10)

['B', 'B', 'G', 'G', 'I', 'F', 'N', 'G', 'J', 'I']
['N', 'G', 'W', 'F', 'N', 'N', 'J', 'F', 'W', 'G']
['W', 'N', 'N', 'G', 'W', 'I', 'N', 'J', 'W', 'F']
['O', 'G', 'J', 'N', 'N', 'N', 'Q', 'F', 'N', 'N']
['W', 'F', 'O', 'G', 'W', 'W', 'N', 'G', 'W', 'I']
['W', 'N', 'W', 'F', 'W', 'N', 'W', 'N', 'N', 'G']
['W', 'F', 'N', 'W', 'J', 'N', 'O', 'G', 'W', 'N']
['O', 'G', 'U', 'F', 'N', 'W', 'W', 'N', 'O', 'G']
['W', 'J', 'N', 'G', 'W', 'N', 'N', 'N', 'W', 'F']
['W', 'N', 'W', 'N', 'N', 'N', 'W', 'F', 'W', 'W']


# Appendix

#### A search on OEIS with SageMath for sequences that are the union of two base sequences.

In [21]:
if 2^3 != 1:
    # This cell (and only this cell) needs the SageMath kernel to run.
    # We use the function 'oeis' from:
    # https://github.com/sagemath/sage/blob/develop/src/sage/databases/oeis.py
    from sage.databases.oeis import oeis
    from itertools import combinations
    import time
    
    SeqPairs = combinations(Anames, 2)

    for a, b in SeqPairs:
        s = sorted([*Aseq[a], *Aseq[b]])
        print((a, b), s[:16])
        print(oeis(s[2:24], 4))
        print()
        time.sleep(2) # to avoid overloading the OEIS server

The output of the SageMath search. Not shown on GitHub because in the 'raw' format, but included in the notebook.

**Part I**: Possible hits, **not** verified:

The output of the SageMath search. Not shown on GitHub because in the 'raw' format, but included in the notebook.

**Part II**: Unions that were not found in the OEIS. We don't think it would be a good idea to include them in the database without a good reason.

####  The Integer Type Tree in LaTeX 

[TikZ code for the Integer Type Tree, ITTree.tex, not shown on GitHub because in 'raw' format, but included in the notebook.]

### The Integer Type Tree in ASCII Art

```
A001477 (Root: Natural numbers)                                               
│── A387487  Idempotents, no prime factors, {0, 1}                            
└── A020725  n >= 2, at least one prime factor                                
    ├── A246655  Prime powers of the form p^k where p prime and k >= 1        
    │   ├── A000040  Primes, one factor with multiplicity = 1                 
    │   │   ├── A002144  Primes = 1 (mod 4) (Pythagorean primes)              
    │   │   └── A045326  Primes = 3 (mod 4), or 2                             
    │   └── A246547  Prime powers p^k where p prime and k >= 2                
    │       ├── A001248  Squares of primes.                                   
    │       └── A246549  Prime powers p^k where p prime and k >= 3            
    └── A024619  Numbers that are not powers of primes p^k (k>=1)             
        ├── A182853  Number of distinct primes > 1, all multiplicities = 1    
        │   ├── A120944  Composite flat numbers                               
        │   │   ├── A006881  Product of two distinct primes                   
        │   │   └── A350352  Products of three or more distinct prime numbers 
        │   └── A303606  Equi-pow composites, all multiplicities equal ≥ 2    
        │       ├── A177492  Products of squares of 2 or more distinct primes 
        │       └── A388304  Prime factors > 1, all exponents are equal > 2   
        └── A059404  Numbers with different multiplicities                    
            ├── A389864  Perfect powers with mixed multiplicities             
            └── A303946  Numbers neither flat nor perfect powers              
                ├── A052486  Powerful but not perfect (Achilles numbers)      
                └── A332785  Nonflat numbers that are not squareful           
```


### Jump table for the tree leaves to the OEIS

| OEIS | Description | First terms |
| --- | --- | --- |
| [A387487](https://oeis.org/A387487) | {0, 1} idempotent numbers; no prime factor | 0, 1 |
| [A045326](https://oeis.org/A045326) | Primes ≡ 3 (mod 4), or 2 | 2, 3, 7, 11, 19, 23, 31, 43, 47, 59, 67, 71, 79, 83, 103, 107, 127, 131, 139, 151 |
| [A001248](https://oeis.org/A001248) | Squares of primes | 4, 9, 25, 49, 121, 169, 289, 361, 529, 841, 961, 1369, 1681, 1849, 2209, 2809, 3481, 3721, 4489, 5041 |
| [A002144](https://oeis.org/A002144) | Primes ≡ 1 (mod 4) (Pythagorean primes) | 5, 13, 17, 29, 37, 41, 53, 61, 73, 89, 97, 101, 109, 113, 137, 149, 157, 173, 181, 193 |
| [A006881](https://oeis.org/A006881) | Product of two distinct primes | 6, 10, 14, 15, 21, 22, 26, 33, 34, 35, 38, 39, 46, 51, 55, 57, 58, 62, 65, 69 |
| [A246549](https://oeis.org/A246549) | Higher prime powers (cubes, fourth powers, …) | 8, 16, 27, 32, 64, 81, 125, 128, 243, 256, 343, 512, 625, 729, 1024, 1331, 2048, 2187, 2197, 2401 |
| [A332785](https://oeis.org/A332785) | Mixed prime powers: some multiplicity ≥ 2, some = 1 | 12, 18, 20, 24, 28, 40, 44, 45, 48, 50, 52, 54, 56, 60, 63, 68, 75, 76, 80, 84 |
| [A350352](https://oeis.org/A350352) | Products of ≥ 3 distinct primes | 30, 42, 66, 70, 78, 102, 105, 110, 114, 130, 138, 154, 165, 170, 174, 182, 186, 190, 195, 210 |
| [A177492](https://oeis.org/A177492) | Products of squares of 2 or more distinct primes | 36, 100, 196, 225, 441, 484, 676, 900, 1089, 1156, 1225, 1444, 1521, 1764, 2116, 2601, 3025, 3249, 3364, 3844 |
| [A052486](https://oeis.org/A052486) | Powerful but not perfect (Achilles numbers) | 72, 108, 200, 288, 392, 432, 500, 648, 675, 800, 864, 968, 972, 1125, 1152, 1323, 1352, 1372, 1568, 1800 |
| [A389864](https://oeis.org/A389864) | Perfect powers with mixed multiplicity | 144, 324, 400, 576, 784, 1600, 1728, 1936, 2025, 2304, 2500, 2704, 2916, 3136, 3600, 3969, 4624, 5184, 5625, 5776 |
| [A388304](https://oeis.org/A388304) | Prime factors > 1, all exponents are equal and > 2 | 216, 1000, 1296, 2744, 3375, 7776, 9261, 10000, 10648, 17576, 27000, 35937, 38416, 39304, 42875, 46656, 50625, 54872, 59319, 74088 |


In [22]:
# OEIS b-file verifier
# (1) Fetches terms from OEIS and saves them in local files
# (2) Uses compute_sequence() to generate terms with the classifier
# (3) Compares computed vs OEIS terms and reports mismatches

import urllib.request
import ssl
import os
import sys

# Clear any potential output buffering
sys.stdout.flush()

print("=== CLEAN VERIFICATION START ===")

ssl_ctx = ssl.create_default_context()

# Core leaf sequences from the Integer Type Tree
TARGET_SEQUENCES = [
    "A045326", "A001248", "A002144", "A006881", 
    "A246549", "A332785", "A350352", "A177492", 
    "A052486", "A389864", "A388304"
]

def parse(data: str, limit: int = 50) -> list[int]:
    # Parse terms
    pairs: list[tuple[int, int]] = []
    for raw in data.splitlines():
        s = raw.strip()
        if not s or s.startswith("#"):
            continue
        s = s.replace(",", " ")
        parts = s.split()
        if len(parts) < 2:
            continue
        try:
            idx = int(parts[0])
            val = int(parts[1])
        except Exception:
            continue
        pairs.append((idx, val))

    if not pairs:
        return []

    pairs.sort(key=lambda t: t[0])
    values = [v for _, v in pairs][:limit]
    return values

def fetch_bfile_from_oeis(seq_id: str) -> bool:
    """Fetch OEIS b-file and save locally."""
    num = seq_id[1:]
    url = f"https://oeis.org/{seq_id}/b{num}.txt"
    filename = f"b{num}.txt"
    
    try:
        with urllib.request.urlopen(url, context=ssl_ctx, timeout=20) as resp:
            data = resp.read().decode("utf-8", "ignore")
    except Exception as e:
        print(f"Warning: Could not fetch {url}: {e}", flush=True)
        return False

    try:
        with open(filename, 'w') as f:
            f.write(data)
        print(f"    Saved {filename}", flush=True)
    except Exception as e:
        print(f"    Warning: Could not save {filename}: {e}", flush=True)
        return False
    return True


def load_local_bfile(seq_id: str, upto: int) -> list[int]:
    """Load terms from local b-file."""
    num = seq_id[1:]
    filename = f"b{num}.txt"
    
    if not os.path.exists(filename):
        print(f"Warning: local file {filename} not found", flush=True)
        return []
    
    try:
        with open(filename, 'r') as f:
            data = f.read()
    except Exception as e:
        print(f"Warning: Could not read {filename}: {e}", flush=True)
        return []
    return parse(data, upto)


def compute_sequence(seq_id: str, count: int) -> list[int]:
    """Compute sequence terms using classifier"""
    results = []
    n = 0
    max_search = 2000000
    progress_interval = 50000
    last_progress = -1  # Track last progress report
    
    while len(results) < count and n < max_search:
        if seq_id == classifier(n):
            results.append(n)
        n += 1
        # Progress indicator - only print once per interval
        if n % progress_interval == 0 and n != last_progress:
            print(f"      ... searched up to n={n}, found {len(results)} terms so far", flush=True)
            last_progress = n
    
    if len(results) < count:
        print(f"      Warning: Only found {len(results)}/{count} terms after searching up to {max_search}", flush=True)
    
    return results


def verify_sequence(seq_id: str, upto: int = 50) -> bool:
    """Verify computed sequence against OEIS b-file."""
    # Step 1: Get OEIS terms
    num = seq_id[1:]
    local_file = f"b{num}.txt"
    
    if os.path.exists(local_file):
        print(f"    Using existing {local_file}", flush=True)
    else:
        print(f"    Fetching {seq_id} from OEIS...", flush=True)
        if not fetch_bfile_from_oeis(seq_id):
            print("Failed to fetch OEIS b-file")
            return False

    oeis_values = load_local_bfile(seq_id, upto)
    if not oeis_values:
        print("Failed to load OEIS b-file")
        return False

    # Step 2: Compute terms using classifier
    print(f"    Computing {seq_id} terms with classifier...", flush=True)
    computed_values = compute_sequence(seq_id, upto)
    print(f"    Found {len(computed_values)} terms for {seq_id}", flush=True)
    
    # Step 3: Compare terms
    compare_count = min(len(oeis_values), len(computed_values), upto)
    oeis_slice = oeis_values[:compare_count]
    computed_slice = computed_values[:compare_count]
    
    if oeis_slice == computed_slice:
        print(f"PASS ({compare_count} terms match)", flush=True)
        return True
    else:
        print(f"FAIL - computed: {computed_slice[:10]}... vs OEIS: {oeis_slice[:10]}...", flush=True)
        return False


def main() -> None:
    print("Starting OEIS b-file verification...\n", flush=True)

    # Setup directories
    os.makedirs("bfiles", exist_ok=True)
    start_dir = os.getcwd()
    os.chdir("bfiles")

    failures = []
    successes = 0
    upto = 50

    for seq in TARGET_SEQUENCES:
        print(f"\nProcessing {seq}:", flush=True)
        passed = verify_sequence(seq, upto)

        if passed:
            print(f"  ✓ {seq}", flush=True)
            successes += 1
        else:
            print(f"  ✗ {seq}", flush=True)
            failures.append(seq)

    # Cleanup
    os.chdir(start_dir)

    print(f"\n=== VERIFICATION SUMMARY ===", flush=True)
    print(f"Total sequences: {len(TARGET_SEQUENCES)}", flush=True)
    print(f"Passed: {successes}", flush=True)
    print(f"Failed: {len(failures)}", flush=True)

    if failures:
        print(f"\nFailures:", flush=True)
        for seq in failures:
            print(f"  {seq}", flush=True)

    print("\n=== VERIFICATION COMPLETE ===", flush=True)

main()

=== CLEAN VERIFICATION START ===
Starting OEIS b-file verification...


Processing A045326:
    Using existing b045326.txt
    Computing A045326 terms with classifier...
    Found 50 terms for A045326
PASS (50 terms match)
  ✓ A045326

Processing A001248:
    Using existing b001248.txt
    Computing A001248 terms with classifier...
      ... searched up to n=50000, found 48 terms so far
    Found 50 terms for A001248
PASS (50 terms match)
  ✓ A001248

Processing A002144:
    Using existing b002144.txt
    Computing A002144 terms with classifier...
    Found 50 terms for A002144
PASS (50 terms match)
  ✓ A002144

Processing A006881:
    Using existing b006881.txt
    Computing A006881 terms with classifier...
    Found 50 terms for A006881
PASS (50 terms match)
  ✓ A006881

Processing A246549:
    Using existing b246549.txt
    Computing A246549 terms with classifier...
      ... searched up to n=50000, found 36 terms so far
      ... searched up to n=100000, found 43 terms so far
     